In [1]:
import sys, datetime
import pandas as pd
import numpy as np

sys.path.insert(1, '../../../../scripts/')
from s3_support import *

In [2]:
form = 787530

In [24]:
q = '''select
            date_trunc('day', date) as day,
            org,
            form,
            sum(views) as pageviews
        from googleanalytics_traffic
            where form={}
        group by date_trunc('day', date), org, form;'''.format(form)
pageviews = redshift_query_read(q)
pageviews = pageviews[pageviews['form']!=0]
pageviews['week'] = pd.to_datetime(pageviews['day'])

In [38]:
q = '''select 
            form, 
            date_trunc('day', date) as day,
            count(id) as count, 
            sum(amount) as vol
        from transactions
        where status='A' and date>=2017 and source='don_form'
            and form={}
        group by form, date_trunc('day', date)'''.format(form)
trans = redshift_query_read(q)
trans['week'] = pd.to_datetime(trans['day'])

In [39]:
len(trans), trans.head()

(27,
      form        day  count    vol       week
 0  787530 2017-09-21      3  699.0 2017-09-21
 1  787530 2017-09-27      2   50.0 2017-09-27
 2  787530 2017-10-02      6  549.0 2017-10-02
 3  787530 2017-10-05      2  149.0 2017-10-05
 4  787530 2017-10-10      4  248.0 2017-10-10)

In [40]:
trans = trans.merge(pageviews, on=['form', 'day'], how='outer')
trans['conversion_rate'] = trans['count'] / trans['pageviews']

In [41]:
#trans = trans.fillna(trans.mean())
trans['new_conversion_rate'] = trans['count'] / trans['pageviews']
trans.head()

,form,day,count,vol,week_x,org,pageviews,week_y,conversion_rate,new_conversion_rate
0,787530,2017-09-21,3.0,699.0,2017-09-21,1229,15,2017-09-21,0.200000,0.200000
1,787530,2017-09-27,2.0,50.0,2017-09-27,1229,131,2017-09-27,0.015267,0.015267
2,787530,2017-10-02,6.0,549.0,2017-10-02,1229,44,2017-10-02,0.136364,0.136364
3,787530,2017-10-05,2.0,149.0,2017-10-05,1229,48,2017-10-05,0.041667,0.041667
4,787530,2017-10-10,4.0,248.0,2017-10-10,1229,72,2017-10-10,0.055556,0.055556


In [42]:
trans['new_conversion_rate'].fillna(0).mean()

0.017257464074561874

In [43]:
pageviews['pageviews'].sum(), trans['pageviews'].sum()

(2720, 2720)

In [44]:
pageviews.head()

,day,org,form,pageviews,week
0,2017-02-21,1229,787530,4,2017-02-21
1,2017-02-22,1229,787530,4,2017-02-22
2,2017-03-16,1229,787530,2,2017-03-16
3,2017-03-18,1229,787530,4,2017-03-18
4,2017-03-19,1229,787530,4,2017-03-19
